In [1]:
import requests, json
from tqdm import tqdm
from bs4 import BeautifulSoup
import urllib.parse as parse
import pandas as pd

In [ ]:
url = 'https://search.naver.com/search.naver?'
qs = 'where=news&sm=tab_pge&query=러시아%20한국%20경제&sort=1&photo=0&field=0&pd=3&ds=2022.02.24&de=2022.04.15&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:from20220224to20220415,a:all&start=2581'
params = dict(parse.parse_qsl(qs))
params['cluster_rank'] = ''
params

'where=news&sm=tab_pge&query=러시아%20한국%20경제&sort=1&photo=0&field=0&pd=3&ds=2022.02.24&de=2022.04.15&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:from20220224to20220415,a:all&start=2581'
'where=news&sm=tab_pge&query=러시아%20한국%20경제&sort=1&photo=0&field=0&pd=3&ds=2022.02.24&de=2022.04.15&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:from20220224to20220415,a:all&start=1'

'where=news&sm=tab_pge&query=러시아%20한국%20경제&sort=1&photo=0&field=0&pd=3&ds=2018.02.24&de=2019.04.15&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:from20180224to20190415,a:all&start=1'

{'cluster_rank': '',
 'de': '2022.04.15',
 'ds': '2022.02.24',
 'field': '0',
 'mynews': '0',
 'nso': 'so:dd,p:from20220224to20220415,a:all',
 'office_section_code': '0',
 'office_type': '0',
 'pd': '3',
 'photo': '0',
 'query': '러시아 한국 경제',
 'sm': 'tab_pge',
 'sort': '1',
 'start': '2581',
 'where': 'news'}

In [2]:
def articlesAdrsLstGen(keywords):
  adrs_lst = []

  for page in tqdm(range(1, 2582, 10)):
    try:
        headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
        }
        url = 'https://search.naver.com/search.naver?'
        qs = 'where=news&sm=tab_pge&query=러시아%20한국%20경제&sort=1&photo=0&field=0&pd=3&ds=2018.02.24&de=2019.04.15&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:dd,p:from20180224to20190415,a:all&start=1'
        params = dict(parse.parse_qsl(qs))
        params['cluster_rank'] = ''
        params['query'] = keywords
        params['start'] = page
        resp = requests.get(url, headers=headers, params=params)
        soup = BeautifulSoup(resp.content, 'lxml')
        adrs = soup.select('div.news_info > div.info_group > a')
        for item in adrs:
          if 'https://news.naver.com/main/read.naver?' in item['href'] and item['href'] not in adrs_lst:
            adrs_lst.append(item['href'])
    except Exception as e:
      print()
      print(e)
      print(item['href'])
      continue
  return adrs_lst

# 'div.news_wrap.api_ani_send > div > div.news_info > div.info_group > a
# '#sp_nws45 > div.news_wrap.api_ani_send > div > div.news_info > div.info_group > a:nth-child(3)'
# '#sp_nws50 > div > div > div.news_info > div.info_group > a:nth-child(3)'

def articlesTableGen(adrs_lst):
  articles = {
    '등록일시': [],
    '제목': [],
    '내용': [],
    '주소': [],
  }

  for adrs in tqdm(adrs_lst):
      try:
        headers = {
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36',
        }
        url = adrs
        resp = requests.get(url, headers=headers)
        soup = BeautifulSoup(resp.content, 'lxml')
        get_title = soup.select('#articleTitle')
        get_time = soup.select('#main_content > div.article_header > div.article_info > div > span.t11')
        get_content = soup.select('#articleBodyContents')
        articles['등록일시'].append(get_time[0].text)
        articles['제목'].append(get_title[0].text)
        articles['내용'].append(get_content[0].text)
        articles['주소'].append(adrs)
      except Exception as e:
        print()
        print(e)
        print('오류주소', adrs)
        continue
  table = pd.DataFrame(articles)
  return table


In [3]:
lst = articlesAdrsLstGen('러시아 한국 경제')

 83%|████████▎ | 214/259 [05:51<01:13,  1.64s/it]


KeyboardInterrupt: ignored

In [ ]:
len(lst)

In [ ]:
len(set(lst))

In [ ]:
lst

In [ ]:
df = articlesTableGen(lst)

  8%|▊         | 113/1456 [01:06<12:35,  1.78it/s]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   등록일시    1000 non-null   object
 1   제목      1000 non-null   object
 2   내용      1000 non-null   object
 3   주소      1000 non-null   object
dtypes: object(4)
memory usage: 31.4+ KB


In [ ]:
df

,등록일시,제목,내용,주소
0,2022.04.14. 오후 12:01,"손경식 경총 회장, 美헤리티지재단 찾아 ""경제협력 강화 기대""",\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
1,2022.04.14. 오후 12:01,"손경식 경총 회장, 美 헤리티지 재단 방문…""韓도 민간 주도 싱크탱크 필요""",\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
2,2022.04.14. 오후 12:01,손경식 경총 회장 “국가 정책 제시하는 민간 싱크탱크 설립 필요”,\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
3,2022.04.14. 오후 12:00,"손경식, 美 싱크탱크 만나 ""한미 경제협력 강화""",\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
4,2022.04.14. 오후 12:00,"美헤리티지 재단 회장 만난 경총 손경식 ""한국형 싱크탱크 필요""",\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
...,...,...,...,...
995,2022.03.17. 오전 10:30,코로나19 재확산에 다시 빗장 건 중국…정부 '공급망 애로해소 창구' 가동,\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
996,2022.03.17. 오전 10:01,"중진공, 중소 백신기업 수출 지원 나서",\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
997,2022.03.17. 오전 9:56,"한미 통상장관, 대러 수출규제·IPEF 등 주요 통상현안 공조 강화 합의",\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...
998,2022.03.17. 오전 9:39,[단독]공급망 충격에 정부·유관기관 논의체 확대…10→25개 늘려 대응,\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction...,https://news.naver.com/main/read.naver?mode=LS...


In [ ]:
cond = df.duplicated()
df[cond].count()

등록일시    0
제목      0
내용      0
주소      0
dtype: int64

In [ ]:
df.to_csv('러시아 한국 경제.csv', index=False)